In [1]:
import torch

import random
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from torch.utils import data
from transformers import DistilBertTokenizer, DistilBertModel
from sklearn.metrics import f1_score, roc_auc_score

SEED = 1234

random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

I0608 17:10:40.846546 4484386240 file_utils.py:39] PyTorch version 1.5.0 available.
I0608 17:10:44.493132 4484386240 file_utils.py:55] TensorFlow version 2.0.0-beta1 available.


In [3]:
df = pd.read_csv("../data/training_data/data1.csv", sep=",")

for i in range(2, 11):
    if i != 3:
        filename = "../data/training_data/data" + str(i) + ".csv"
        df = df.append(pd.read_csv(filename, sep=","))
import math

df = df.sample(frac=1)

train_num = math.ceil(0.7 * len(df))
valid_num = math.ceil(0.9 * len(df))
train_data = df.iloc[:train_num, :].reset_index()
valid_data = df.iloc[train_num:valid_num, :].reset_index()
test_data = df.iloc[valid_num:, :].reset_index()

In [8]:
df.head()

,Unnamed: 0,tweet_id,sentiment,text
1834,1834,6.244020e+17,positive,"A great crowd of nearly 6,000 enjoyed National..."
7119,7119,1.187997e+17,neutral,"Andy Rooney, 92, will make last appearance on ..."
14328,14328,6.407009e+17,neutral,[someone] are you going over auntie nicki's to...
9429,9429,2.640793e+17,neutral,i may or may not be obsessed with Poetic Justice.
7033,7033,6.345089e+17,negative,Caitlyn Jenner may be charged with manslaughte...


In [4]:
num_positive = (df["sentiment"] == "positive").sum()
num_negative = (df["sentiment"] == "negative").sum()
num_neutral = (df["sentiment"] == "neutral").sum()

args["weight"] = torch.tensor([num_negative / len(df), num_neutral / len(df), num_positive / len(df)], dtype=torch.float32)

print(args["weight"])

tensor([0.1568, 0.4639, 0.3793])


In [5]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')

I0608 17:11:41.010191 4484386240 tokenization_utils.py:1022] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at /Users/raina/.cache/torch/transformers/26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084


In [6]:
tokenized_train = train_data['text'].apply((
    lambda x: tokenizer.encode(x, add_special_tokens=True)))
tokenized_valid = valid_data['text'].apply((
    lambda x: tokenizer.encode(x, add_special_tokens=True)))
tokenized_test = test_data['text'].apply((
    lambda x: tokenizer.encode(x, add_special_tokens=True)))

In [7]:
def get_max_len(tokenized):
    max_len = 0
    for i in tokenized.values:
        if len(i) > max_len:
            max_len = len(i)
    return max_len